In [0]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import pathlib
import os
import boto3

# Define paths for your custom dataset
aluminum_path = ["/Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/", 
                "/Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/real_world/"] 

foodWaste_path = ["/Workspace/Shared/dataset/AI-GarbagePhotos/food_waste/default/", 
                  "/Workspace/Shared/dataset/AI-GarbagePhotos/food_waste/real_world/"]

paper_path = ["/Workspace/Shared/dataset/AI-GarbagePhotos/paper_cups/default/", 
              "/Workspace/Shared/dataset/AI-GarbagePhotos/paper_cups/real_world/"]

plastic_path = ["/Workspace/Shared/dataset/AI-GarbagePhotos/plastic_water_bottles/default/", 
                "/Workspace/Shared/dataset/AI-GarbagePhotos/plastic_water_bottles/real_world/"]

# Define image size and number of classes (adjust based on your dataset)
img_width = 128  # Resize to match your dataset's image size
img_height = 128  # Resize to match your dataset's image size
num_classes = 4  # aluminum, food waste, paper, plastic


full_path = '/Workspace/Shared/dataset/AI-GarbagePhotos/'

if os.path.exists(full_path):
    # List files in the directory
    print("Directory exists. Contents:")
    for filename in os.listdir(full_path):
        print(filename)
else:
    print("Directory does not exist.")
# --- Now let's handle your custom dataset ---
# Function to load and preprocess images
def load_and_preprocess_image(image_path, img_width=128, img_height=128):
    img = tf.io.read_file(image_path)
    try:
        img = tf.image.decode_jpeg(img, channels=3)  # JPEG format
    except tf.errors.InvalidArgumentError:
        try:
            img = tf.image.decode_png(img, channels=3)  # PNG format
        except tf.errors.InvalidArgumentError:
            raise ValueError(f"Unsupported image format for file {image_path}")

    img = tf.image.resize(img, [img_width, img_height])
    img = img / 255.0  # Normalize to [0, 1]
    return img

# Function to load dataset from paths
def load_custom_dataset(paths, img_width=128, img_height=128):
    image_paths = []
    labels = []

    label_dict = {
        "aluminum_soda_cans": 0,
        "food_waste": 1,
        "paper_cups": 2,
        "plastic_water_bottles": 3
    }

    for path in paths:
        path_obj = pathlib.Path(path)
        print("path object = ", path_obj)
        category_name = path_obj.parent.name
        print("category_name =", category_name)
        if category_name not in label_dict:
            raise ValueError(f"Unexpected folder name '{category_name}' in path: {path}")
        
        label = label_dict[category_name]
        print("label", label)
        # Loop through image files and append paths and labels
        image_files_list = path_obj.rglob('*.png') # Adjust file format if necessary
        image_list =list(image_files_list)
        for image_file in image_list:
            print("image_name =", image_file)
            image_paths.append(str(image_file))
            labels.append(label)

    # Debugging: Check the contents of image_paths and labels
    # print(f"Total images: {len(image_paths)}")
    # print(f"First 5 image paths: {image_paths[:5]}")
    # print(f"First 5 labels: {labels[:5]}")

    if len(image_paths) == 0:
        raise ValueError("No images found in dataset!")

    return image_paths, labels


# Load custom dataset paths for aluminum, food waste, paper, and plastic
image_paths, labels = load_custom_dataset(aluminum_path + foodWaste_path + paper_path + plastic_path)
print(labels)
# Convert to TensorFlow Dataset for batch processing
def create_tf_dataset(image_paths, labels, batch_size=16):
    if not image_paths or not labels:
        raise ValueError("Image paths or labels are empty!")

    path_ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    def load_fn(image_path, label):
        img = load_and_preprocess_image(image_path)
        return img, label
    
    dataset = path_ds.map(load_fn).batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


# Create a TensorFlow dataset
custom_dataset = create_tf_dataset(image_paths, labels)

# Define a new model for your custom dataset (4 classes: aluminum, food waste, paper, plastic)
model_custom = tf.keras.models.Sequential([
    Input(shape=(128, 128, 3)),  # Adjust the shape as necessary
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Assuming 4 classes
])

# Compile the model
model_custom.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',  # Sparse because labels are integers
                     metrics=['accuracy'])

# Train the model on the custom dataset
model_custom.fit(custom_dataset, epochs=5)

# You can evaluate the model on a custom test dataset in a similar manner
import tensorflow as tf
import numpy as np

def preprocess_and_predict(image_path, model, img_height=128, img_width=128):
    # Load the image file
    img_raw = tf.io.read_file(image_path)
    # Decode it into a tensor
    img_tensor = tf.image.decode_image(img_raw, channels=3, expand_animations=False)
    # Resize it to the required size
    img_resized = tf.image.resize(img_tensor, [img_height, img_width])
    # Normalize the image to the range [0, 1]
    img_normalized = img_resized / 255.0
    # Add a batch dimension
    img_batch = tf.expand_dims(img_normalized, axis=0)

    # Predict the class of the image
    predictions = model.predict(img_batch)
    predicted_class_id = np.argmax(predictions, axis=1)[0]
    class_names = ['aluminum', 'food waste', 'paper', 'plastic']
    predicted_class_name = class_names[predicted_class_id]
    
    return predicted_class_name

# Example usage
image_path = '/Workspace/Shared/test_data/test_plastic_bottle.jpg'  # Path to the new image to classify
predicted_category = preprocess_and_predict(image_path, model_custom)
print("The uploaded image is classified as:", predicted_category)


Directory exists. Contents:
aluminum_soda_cans
food_waste
paper_cups
plastic_water_bottles
path object =  /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default
category_name = aluminum_soda_cans
label 0
image_name = /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/Image_25.png
image_name = /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/Image_26.png
image_name = /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/Image_27.png
image_name = /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/Image_3.png
image_name = /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/Image_4.png
image_name = /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/Image_5.png
image_name = /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/Image_6.png
image_name = /Workspace/Shared/dataset/AI-GarbagePhotos/aluminum_soda_cans/default/Image_60.png
image_name = /Wor

2024-11-10 13:13:52.286103: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 12s 285ms/step - accuracy: 0.6537 - loss: 7.3257
Epoch 2/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 266ms/step - accuracy: 0.4989 - loss: 2.0664
Epoch 3/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 269ms/step - accuracy: 0.2987 - loss: 1.4789
Epoch 4/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 10s 264ms/step - accuracy: 0.3557 - loss: 1.7121
Epoch 5/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 10s 257ms/step - accuracy: 0.3095 - loss: 1.5857
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


2024-11-10 13:14:45.711566: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
The uploaded image is classified as: food waste
